In [1]:
import numpy as np
import imageio
from scipy import ndimage

In [2]:
def smooth_gaussian(im, sigma):

    if sigma == 0:
        return im

    im_smooth = im.astype(float)
    kernel_x = np.arange(-3*sigma,3*sigma+1).astype(float)
    kernel_x = np.exp((-(kernel_x**2))/(2*(sigma**2)))

    im_smooth = scipy.ndimage.convolve(im_smooth, kernel_x[np.newaxis])

    im_smooth = scipy.ndimage.convolve(im_smooth, kernel_x[np.newaxis].T)

    return im_smooth

In [3]:
def gradient(im_smooth):

    gradient_x = im_smooth.astype(float)
    gradient_y = im_smooth.astype(float)

    kernel = np.arange(-1,2).astype(float)
    kernel = - kernel / 2

    gradient_x = scipy.ndimage.convolve(gradient_x, kernel[np.newaxis])
    gradient_y = scipy.ndimage.convolve(gradient_y, kernel[np.newaxis].T)

    return gradient_x,gradient_y

In [4]:
def sobel(im_smooth):
    gradient_x = im_smooth.astype(float)
    gradient_y = im_smooth.astype(float)

    kernel = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])

    gradient_x = ndimage.convolve(gradient_x, kernel)
    gradient_y = ndimage.convolve(gradient_y, kernel.T)

    return gradient_x,gradient_y

In [5]:
def compute_normal_map(gradient_x, gradient_y, intensity=1):

    width = gradient_x.shape[1]
    height = gradient_x.shape[0]
    max_x = np.max(gradient_x)
    max_y = np.max(gradient_y)

    max_value = max_x

    if max_y > max_x:
        max_value = max_y

    normal_map = np.zeros((height, width, 3), dtype=np.float32)

    intensity = 1 / intensity

    strength = max_value / (max_value * intensity)

    normal_map[..., 0] = gradient_x / max_value
    normal_map[..., 1] = gradient_y / max_value
    normal_map[..., 2] = 1 / strength

    norm = np.sqrt(np.power(normal_map[..., 0], 2) + np.power(normal_map[..., 1], 2) + np.power(normal_map[..., 2], 2))

    normal_map[..., 0] /= norm
    normal_map[..., 1] /= norm
    normal_map[..., 2] /= norm

    normal_map *= 0.5
    normal_map += 0.5

    return normal_map

In [7]:
sigma = 0 #smooth gussion blur applied to the image
intensity = 1.5 #intensity of the normal map
input_file = "Door3-large.png" 
output_file = "Door3_nm.png"

im = imageio.imread(input_file)

if im.ndim == 3:
    im_grey = np.zeros((im.shape[0],im.shape[1])).astype(float)
    im_grey = (im[...,0] * 0.3 + im[...,1] * 0.6 + im[...,2] * 0.1)
    im = im_grey

im_smooth = smooth_gaussian(im, sigma)

sobel_x, sobel_y = sobel(im_smooth)

normal_map = compute_normal_map(sobel_x, sobel_y, intensity)

imageio.imsave(output_file, normal_map)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [90]:
def edge(im, y, x, height, width):
    #    a b c y=0
    #    d e f y=1
    #    g h i y=2
    #x = 0 1 2
    norm_front = [0, 0, -1]
    norm_up = [0, 1, 0]
    norm_right_up = [1, 1, 0]
    norm_right = [1, 0, 0]
    norm_right_down = [1, -1, 0]
    norm_down = [0, -1, 0]
    norm_left_down = [-1, -1, 0]
    norm_left = [-1, 0, 0]
    norm_left_up = [-1, 1, 0]
    
    #image edge
    if x == 0:
        return norm_left
    if y == 0:
        return norm_up
    
    if x == width - 1:
        return norm_right
    if y == height - 1: 
        return norm_down
    
    
    try:
        a = im[y-1][x-1][3]/255 #alpha
        b = im[y-1][x][3]/255
        c = im[y-1][x+1][3]/255
        d = im[y][x-1][3]/255
        e = im[y][x][3]/255
        f = im[y][x+1][3]/255
        g = im[y+1][x-1][3]/255
        h = im[y+1][x][3]/255
        i = im[y+1][x+1][3]/255
        
        
        up = a + b + c 
        down = g + h + i
        left = a + d + g
        right = c + f + i

        
        #blank
        if up == down == left == right == 0:
            return norm_front
        
        #up_edge
        if up == 0:
            if left == 0 or right == 0:
                if left ==0:
                    return norm_left_up
                else:
                    return norm_right_up
            else:  
                return norm_up
            
        #down_edge
        if down == 0:
            if left == 0 or right == 0:
                if left == 0:
                    return norm_left_down
                else:
                    return norm_right_down
            else:
                return norm_down
        
        
        #left_edge
        if left == 0:
            return norm_left
        #right_edge
        if right == 0:
            return norm_right
        
        
        return norm_front     
        
        
    except Exception as e:
        print(str(e), y, x)
        return norm_front
        

In [91]:
def int2Color(v):
    r = v[0]
    g = v[1]
    b = v[2]
    return r*127+128, g*127+128, 128-b*127

In [93]:
input_file = "Door3-large.png" 
output_file = "Door3_edge2.png"

im = imageio.imread(input_file)
(height, width, channel) = im.shape
print(im.shape)
normal_map = np.zeros((height, width, 3), dtype=np.uint8)
for x in range(width):
    for y in range(height):    
        normal_map[y, x] = int2Color(edge(im, y, x, height, width))

imageio.imsave(output_file, normal_map)
print('done')

(530, 328, 4)
done
